<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Collect-S&amp;P-500-Companies" data-toc-modified-id="Collect-S&amp;P-500-Companies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Collect S&amp;P 500 Companies</a></span></li><li><span><a href="#Example-code" data-toc-modified-id="Example-code-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Example code</a></span></li><li><span><a href="#Stock-Prices" data-toc-modified-id="Stock-Prices-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Stock Prices</a></span></li><li><span><a href="#Calculate-Correlation" data-toc-modified-id="Calculate-Correlation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Calculate Correlation</a></span></li></ul></div>

In [48]:
# Import libraries 
import pandas as pd
import os

from datetime import datetime
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from urllib.parse import urlparse

%matplotlib inline

# Collect S&P 500 Companies

In [2]:
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sandp_df = table[0]

#sandp_df.to_csv('data/S&P500-Info.csv')
#sandp_df.to_csv("data/S&P500-Symbols.csv", columns=['Symbol'])

#https://medium.com/wealthy-bytes/5-lines-of-python-to-automate-getting-the-s-p-500-95a632e5e567

In [3]:
sandp_df.head(5)
# so the symbol is the same as the corresponding stock ticker. 
# It will be used for parsing news results that reference the company that made the headlines.

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


# Example code

Taken from https://towardsdatascience.com/sentiment-analysis-of-stocks-from-financial-news-using-python-82ebdcefb638

In [4]:
finwiz_url = 'https://finviz.com/quote.ashx?t='

news_tables = {}
tickers = sandp_df['Symbol']

for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url, headers={'user-agent': 'my-app/0.0.1'}) 
    try:
        response = urlopen(req)   
        html = BeautifulSoup(response)  # Read the contents of the file into 'html'
        news_table = html.find(id='news-table') # Find 'news-table' in the Soup and load it into 'news_table'
        news_tables[ticker] = news_table # Add the table to our dictionary
    except:
        continue

In [53]:
# define function to generate [ticker, date, time, headline, news_source] given a article
def article_summary(html_article):
    # read the text from each article into text
        # get text from a only
        headline = html_article.a.get_text() 
        # splite text in the td tag into a list 
        date_scrape = html_article.td.text.split()
        
        # get news media company
        news = html_article.span.get_text()
        
        global date
        # if the length of 'date_scrape' is 1, load 'time' as the only element
        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        # else load 'date' as the 1st element and 'time' as the second    
        else:
            date = date_scrape[0]
            time = date_scrape[1]
        
        # Return [ticker, date, time, headline, news source] to be appended
        return [date, time, headline, news]

    
# define function to generate article content (in a list) given a article
# only works with articles from yahoo.finance right now
def article_content(html_article):
    # get the link to the full article
    link = html_article.find('a').get('href')
    content = 'empty string'
    # check if link leads to yahoo.finance
    if urlparse(link).netloc == 'finance.yahoo.com':
        try:
            # request from yahoo.finance
            req_art = Request(url=link, headers={'user-agent':'my-app/0.0.1'})
            response_art = urlopen(req_art)
            html_art = BeautifulSoup(response_art)
            # get the article content
            content = html_art.find(class_='caas-body').get_text()
        except:
            print('Error following article link: ', link)
    else:
        return [content, urlparse(link).netloc]
    return [content]
    
    

In [71]:
# Define function to prepare data with ticker, date, time, headline, news, content
def prepare_data(html_dict):
    
    pid = os.get_pid()
    
    # define where data will be saved
    global parsed_news
    global rejected_sites
    parsed_news = []
    rejected_sites = []
    
    for file_name, news_str in html_dict.items():
        # soupify str
        news_html = BeautifulSoup(news_str, 'html.parser')
        # iterate through all tr tags in 'news_table'
        for x in news_html.findAll('tr'):
            # article content and rejected site (if applicable) (use to expand available sites in the future)
            art_content = article_content(x)
            ticker = file_name.split('_')[0]
            # combine article summary and article content into 1 row of info about the article
            art_row = [ticker] + article_summary(x) + [art_content[0]]
            # append relevant info to the 'parsed_news' list
            parsed_news.append(art_row)
            if len(art_content) > 1:
                rejected_sites.append(art_content[1])
        print('Done parsing through: ', ticker)
    return pid

In [39]:
#print(test_news_tables['MMM'])
print(pd.DataFrame.from_dict(test_news_tables, orient='index').iloc[0, 1])

<tr><td align="right" style="white-space:nowrap" width="130">Feb-14-21 09:27AM  </td><td align="left"><div class="news-link-container"><div class="news-link-left"><a class="tab-link-news" href="https://www.fool.com/investing/2021/02/14/these-3-dividend-stocks-are-too-cheap-to-ignore/?source=eptyholnk0000202&amp;utm_source=yahoo-host&amp;utm_medium=feed&amp;utm_campaign=article" target="_blank">These 3 Dividend Stocks Are Too Cheap to Ignore</a></div><div class="news-link-right"><span style="color:#aa6dc0;font-size:9px"> Motley Fool</span></div></div></td></tr>


In [70]:
# Process Pool Executor

from loky import get_reusable_executor
import time



start_test1 = time.perf_counter()



if __name__ == '__main__':
    test_news_tables = news_tables.copy()
    test_news_tables = dict(zip(['MMM', 'ABT', 'ABBV'], [str(test_news_tables[x]) for x in ['MMM', 'ABT', 'ABBV']]))
    executor = get_reusable_executor(max_workers=4, timeout=2)
    p = executor.submit(prepare_data, test_news_tables)
    print('Processor ID: ', p.result())
        
        
columns = ['ticker', 'date', 'time', 'headline', 'news', 'content']

# Convert the parsed_news list into a DataFrame called 'parsed_news_updated'
parsed_news_updated = pd.DataFrame(parsed_news, columns=columns)
ori_len = len(parsed_news_updated)
# remove articles whose contents are not available
parsed_news_updated_cont = parsed_news_updated.copy()
parsed_news_updated_cont = parsed_news_updated_cont.loc[parsed_news_updated_cont['content'] != 'empty string']

parsed_news_updated = parsed_news_updated.drop('content', axis=1)

print('Percentage decrease in articles is: ', (ori_len - len(parsed_news_updated_cont))/ori_len)


print(parsed_news_updated_cont.head(10))
print(rejected_sites[:20])

end_test1 = time.perf_counter()

print('Process Pool Executor finished in: ', end_test1 - start_test1, ' seconds')

Processor ID:  Processing Complete
Percentage decrease in articles is:  0.31666666666666665
   ticker       date     time  \
1     MMM  Feb-12-21  05:12PM   
5     MMM  Feb-09-21  02:35PM   
6     MMM  Feb-09-21  11:07AM   
7     MMM  Feb-09-21  11:07AM   
9     MMM  Feb-06-21  09:12AM   
12    MMM  Feb-06-21  02:53AM   
13    MMM  Feb-04-21  10:05AM   
14    MMM  Feb-04-21  08:25AM   
15    MMM  Feb-02-21  07:53PM   
16    MMM  Feb-02-21  07:15PM   

                                             headline              news  \
1     First Eagle Investment's Top 4th-Quarter Trades     GuruFocus.com   
5               3M Announces Upcoming Investor Events       PR Newswire   
6   New 3M Polisher ST reduces the number of bioph...       PR Newswire   
7   New 3M Polisher ST reduces the number of bioph...         CNW Group   
9                30 Dividend Kings of 2021 (Part III)    Insider Monkey   
12  Income Investors Should Know That 3M Company (...   Simply Wall St.   
13  3M and Discover

In [69]:
# Thread Pool Executor

from concurrent.futures import ThreadPoolExecutor
import time



start_test1 = time.perf_counter()



if __name__ == '__main__':
    test_news_tables = news_tables.copy()
    test_news_tables = dict(zip(['MMM', 'ABT', 'ABBV'], [str(test_news_tables[x]) for x in ['MMM', 'ABT', 'ABBV']]))
    executor = ThreadPoolExecutor()
    p = executor.submit(prepare_data, test_news_tables)
    print(p.result())
        
        
columns = ['ticker', 'date', 'time', 'headline', 'news', 'content']

# Convert the parsed_news list into a DataFrame called 'parsed_news_updated'
parsed_news_updated = pd.DataFrame(parsed_news, columns=columns)
ori_len = len(parsed_news_updated)
# remove articles whose contents are not available
parsed_news_updated_cont = parsed_news_updated.copy()
parsed_news_updated_cont = parsed_news_updated_cont.loc[parsed_news_updated_cont['content'] != 'empty string']

parsed_news_updated = parsed_news_updated.drop('content', axis=1)

print('Percentage decrease in articles is: ', (ori_len - len(parsed_news_updated_cont))/ori_len)


print(parsed_news_updated_cont.head(10))
print(rejected_sites[:20])

end_test1 = time.perf_counter()

print('Thread Pool Executor finished in: ', end_test1 - start_test1, ' seconds')

Done parsing through:  MMM
Done parsing through:  ABT
Done parsing through:  ABBV
Processing Complete
Percentage decrease in articles is:  0.31666666666666665
   ticker       date     time  \
1     MMM  Feb-12-21  05:12PM   
5     MMM  Feb-09-21  02:35PM   
6     MMM  Feb-09-21  11:07AM   
7     MMM  Feb-09-21  11:07AM   
9     MMM  Feb-06-21  09:12AM   
12    MMM  Feb-06-21  02:53AM   
13    MMM  Feb-04-21  10:05AM   
14    MMM  Feb-04-21  08:25AM   
15    MMM  Feb-02-21  07:53PM   
16    MMM  Feb-02-21  07:15PM   

                                             headline              news  \
1     First Eagle Investment's Top 4th-Quarter Trades     GuruFocus.com   
5               3M Announces Upcoming Investor Events       PR Newswire   
6   New 3M Polisher ST reduces the number of bioph...       PR Newswire   
7   New 3M Polisher ST reduces the number of bioph...         CNW Group   
9                30 Dividend Kings of 2021 (Part III)    Insider Monkey   
12  Income Investors Should

In [68]:
import time


start_test2 = time.perf_counter()


test_news_tables = news_tables.copy()
test_news_tables = dict(zip(['MMM', 'ABT', 'ABBV'], [str(test_news_tables[x]) for x in ['MMM', 'ABT', 'ABBV']]))
print(prepare_data(test_news_tables))

# Set column names
columns = ['ticker', 'date', 'time', 'headline', 'news', 'content']

# Convert the parsed_news list into a DataFrame called 'parsed_and_scored_news'
parsed_news_updated = pd.DataFrame(parsed_news, columns=columns)
ori_len = len(parsed_news_updated)
# remove articles whose contents are not available
parsed_news_updated_content = parsed_news_updated.copy()
parsed_news_updated_content = parsed_news_updated_content.loc[parsed_news_updated['content'] != 'empty string']
parsed_news_updated = parsed_news_updated.drop('content', axis=1)
print('Percentage decrease in articles is: ', (ori_len - len(parsed_news_updated_content))/ori_len)


print(parsed_news_updated_content.head(10))
print(rejected_sites[:20])


end_test2 = time.perf_counter()

print('Synchronous test finished in: ', end_test2 - start_test2, ' seconds')

Done parsing through:  MMM
Done parsing through:  ABT
Done parsing through:  ABBV
Processing Complete
Percentage decrease in articles is:  0.31666666666666665
   ticker       date     time  \
1     MMM  Feb-12-21  05:12PM   
5     MMM  Feb-09-21  02:35PM   
6     MMM  Feb-09-21  11:07AM   
7     MMM  Feb-09-21  11:07AM   
9     MMM  Feb-06-21  09:12AM   
12    MMM  Feb-06-21  02:53AM   
13    MMM  Feb-04-21  10:05AM   
14    MMM  Feb-04-21  08:25AM   
15    MMM  Feb-02-21  07:53PM   
16    MMM  Feb-02-21  07:15PM   

                                             headline              news  \
1     First Eagle Investment's Top 4th-Quarter Trades     GuruFocus.com   
5               3M Announces Upcoming Investor Events       PR Newswire   
6   New 3M Polisher ST reduces the number of bioph...       PR Newswire   
7   New 3M Polisher ST reduces the number of bioph...         CNW Group   
9                30 Dividend Kings of 2021 (Part III)    Insider Monkey   
12  Income Investors Should

In [ ]:
parsed_news = []
rejected_sites = []

# Iterate through the news
for file_name, news_table in news_tables.items():
    # Iterate through all tr tags in 'news_table'
    for x in news_table.findAll('tr'):
        art_content = article_content(x)
        # combine article summary and article content into 1 row of info about the article
        article_row = article_summary(x) + [art_content[0]]
        # Append relevant info as a list to the 'parsed_news' list
        parsed_news.append(article_row)
        rejected_sites.append(art_content[1])

# Set column names
columns = ['ticker', 'date', 'time', 'headline', 'news', 'content']

# Convert the parsed_news list into a DataFrame called 'parsed_and_scored_news'
parsed_news_updated = pd.DataFrame(parsed_news, columns=columns)
ori_len = len(parsed_news_updated)
# remove articles whose contents are not available
parsed_news_updated_content = parsed_news_updated.copy()
parsed_news_updated_content = parsed_news_updated_content.loc[parsed_news_updated['content'] != 'empty string']
parsed_news_updated = parsed_news_updated.drop('content', axis=1)
print('Percentage decrease in articles is: ', (ori_len - len(parsed_news_updated_content))/ori_len)
parsed_news_updated_content.head(10)

#parsed_news_updated does not have the content column and has full number of articles (100 per ticker)
#parsed_news_updated_content has content column and is shortened to remove all rows with no content

In [ ]:
# Count number of headlines produced by each news source and remove news sources with <median headlines
parsed_news_updated = pd.DataFrame(parsed_news, columns=columns)
# generate series of counts
news_count = parsed_news_updated['news'].value_counts()
# set minimum count = median
cutoff_point = news_count.median()
# append count of news to dataframe
parsed_news_updated = parsed_news_updated.merge(news_count, left_on='news', right_index=True)
parsed_news_updated = parsed_news_updated.drop('news_x', axis=1).rename({'news_y' : 'count'}, axis=1)
# remove news which have count < cutoff_point
parsed_news_updated = parsed_news_updated.loc[parsed_news_updated['count'] > cutoff_point]
parsed_news_updated


In [ ]:
#need to tokenize each words within the headlines to improve the sentiment score.

import re
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def regex(x):
    special_chars_p = "[.®'&$’\"\-()#@!?/:]"
    s1 = re.sub(special_chars_p, '', x)  
    return(s1)

parsed_news_updated['headline'] = parsed_news_updated['headline'].apply(regex)

stemmer = PorterStemmer()

def stem_sentences(sentence):
    tokens = sentence.lower().split()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

parsed_news_updated_stem = parsed_news_updated.copy()
parsed_news_updated_stem['headline'] = parsed_news_updated_stem['headline'].apply(stem_sentences)

stop=stopwords.words('english')

parsed_news_updated['headline'].apply(lambda x: [item for item in x if item not in stop])
parsed_news_updated_stem['headline'].apply(lambda x: [item for item in x if item not in stop])

parsed_news_updated['headline'] = parsed_news_updated['headline'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)])) 
parsed_news_updated_stem['headline'] = parsed_news_updated_stem['headline'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)])) 
parsed_news_updated.head()

In [ ]:
# NLTK VADER for sentiment analysis (unstem)
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Instantiate the sentiment intensity analyzer
vader = SentimentIntensityAnalyzer()

# Iterate through the headlines and get the polarity scores using vader
scores = parsed_news_updated['headline'].apply(vader.polarity_scores).tolist()

# Convert the 'scores' list of dicts into a DataFrame
scores_df = pd.DataFrame(scores)

# Join the DataFrames of the news and the list of dicts
parsed_and_scored_news = parsed_news_updated.join(scores_df, rsuffix='_right')

# Convert the date column from string to datetime
parsed_and_scored_news['date'] = pd.to_datetime(parsed_and_scored_news.date).dt.date

parsed_and_scored_news.head(10)

In [ ]:
# NLTK VADER for sentiment analysis (stem)

# Instantiate the sentiment intensity analyzer
vader = SentimentIntensityAnalyzer()

# Iterate through the headlines and get the polarity scores using vader
scores_stem = parsed_news_updated_stem['headline'].apply(vader.polarity_scores).tolist()

# Convert the 'scores' list of dicts into a DataFrame
scores_stem_df = pd.DataFrame(scores_stem)

# Join the DataFrames of the news and the list of dicts
parsed_and_scored_news_stem = parsed_news_updated_stem.join(scores_stem_df, rsuffix='_right')

# Convert the date column from string to datetime
parsed_and_scored_news_stem['date'] = pd.to_datetime(parsed_and_scored_news_stem.date).dt.date

parsed_and_scored_news_stem.head(10)

# Stock Prices

In [ ]:
# Get S&P 500 prices
# source: https://www.spglobal.com/spdji/en/indices/equity/sp-500/#overview

df_sp = pd.read_csv('data/S&P500_5years.csv', usecols=[0,1]) # Use only first 2 columns
df_sp.columns = ['date', 'price']
df_sp['date'] = pd.to_datetime(df_sp['date'])
df_sp.head()

In [ ]:
# Get S&P 500 individual stock prices

# Create a function to get stock price given a ticker 
def get_stock_price(ticker, start, end):
    '''Get prices of a stock in a given period.
    
    Args:
        ticker (str): ticker of a company 
        start (str): date in format of 'YYYY-MM-DD'
        end (str): date in format of 'YYYY-MM-DD'
    
    Returns:
        A DataFrame containing open, high, low, close, volume, dividends, stock splits
    '''
    import yfinance as yf
    
    ticker = yf.Ticker(ticker)
    data = ticker.history(start=start, end=end)
    data.reset_index(level=0, inplace=True)
    return data 


In [ ]:
# Define function to generate merged dataframe (merged by date)
# columns: ['ticker', 'date', 'time', 'headline', 'news', 'neg', 'neu', 'pos', 'compound', 'open', 'close', 'change']
# scored dataframe should be the input (not sure if works for scored sentiment other than Vader)


def generate_final_df(scored_df):
    # Get a list of 505 stocks from S&P 500
    sp500 = sandp_df['Symbol'].unique()
    start = scored_df['date'].min()
    end = scored_df['date'].max()
    
    # Iterate through each stock to get price
    df_stock = pd.DataFrame()
    for ticker in sp500:
        data = get_stock_price(ticker, start, end)
        data['ticker'] = ticker
        df_stock = pd.concat([df_stock, data], axis=0)
        
    # Change all columns names to lowercase  
    df_stock.columns = df_stock.columns.str.lower()
    
    # Convert timestamp to date
    df_stock['date'] = df_stock['date'].apply(datetime.date)
    
    # Reset index
    df_stock.reset_index(drop=True, inplace=True)
    
    # Merge stock price info and sentiment scores
    df_merged = scored_df.merge(df_stock.loc[:, ['date', 'ticker', 'open', 'close']], on=['date', 'ticker'])
    # Add column: price change
    df_merged['change'] = df_merged['close'] - df_merged['open']
    
    return df_merged

In [ ]:
# Generate final df for unstem and stem

df_final_unstem = generate_final_df(parsed_and_scored_news)
df_final_stem = generate_final_df(parsed_and_scored_news_stem)

# Calculate Correlation 

In [ ]:
# Calculate pearson correlation coef between sentiment score and price for each news media
scores_close_unstem = df_final_unstem.groupby('news')[['compound', 'close']].corr().unstack().iloc[:, 1].sort_values(ascending=False)
scores_close_stem = df_final_stem.groupby('news')[['compound', 'close']].corr().unstack().iloc[:, 1].sort_values(ascending=False)
scores_change_unstem = df_final_unstem.groupby('news')[['compound', 'change']].corr().unstack().iloc[:, 1].sort_values(ascending=False)
scores_change_stem = df_final_stem.groupby('news')[['compound', 'change']].corr().unstack().iloc[:, 1].sort_values(ascending=False)


# https://stackoverflow.com/questions/28988627/pandas-correlation-groupby

In [ ]:
# Calculate spearman correlation coef between sentiment score and price for each news media
scores_close_unstem = df_final_unstem.groupby('news')[['compound', 'close']].corr(method='spearman').unstack().iloc[:, 1].sort_values(ascending=False)
scores_close_stem = df_final_stem.groupby('news')[['compound', 'close']].corr(method='spearman').unstack().iloc[:, 1].sort_values(ascending=False)
scores_change_unstem = df_final_unstem.groupby('news')[['compound', 'change']].corr(method='spearman').unstack().iloc[:, 1].sort_values(ascending=False)
scores_change_stem = df_final_stem.groupby('news')[['compound', 'change']].corr(method='spearman').unstack().iloc[:, 1].sort_values(ascending=False)


In [ ]:
# Pearson's Correlation Coefficient as Dataframe
pearson_corr = pd.DataFrame({'variable' : ['close', 'change'], 'unstem' : [df_final_unstem[['compound', 'close']].corr().iloc[0,1], df_final_unstem[['compound', 'change']].corr().iloc[0,1]], 'stem' : [df_final_stem[['compound', 'close']].corr().iloc[0,1], df_final_stem[['compound', 'change']].corr().iloc[0,1]]})
pearson_corr

In [ ]:
# Spearman's rank correlation
spearman_corr = pd.DataFrame({'variable' : ['close', 'change'], 'unstem' : [df_final_unstem[['compound', 'close']].corr(method='spearman').iloc[0,1], df_final_unstem[['compound', 'change']].corr(method='spearman').iloc[0,1]], 'stem' : [df_final_stem[['compound', 'close']].corr(method='spearman').iloc[0,1], df_final_stem[['compound', 'change']].corr(method='spearman').iloc[0,1]]})
spearman_corr

Vader Sentiment Analysis

Performed Pearson's Correlation Coefficient comparisons between compound sentiment and (1) closing price (2) change in price (closing price - opening price). Computed the Spearman rank correlation coefficient as well.

Compared effect of stemming and not stemming words on Pearson's and Spearman's correlation coefficient.


Conclusion:

Pearson:
Correlation between both (1) and (2) is negligible (<1%) without stemming. Stemming appears to improve correlation, but correlation is still very small (<2%)

Spearman:
Correlation for both (1) and (2) is still small, but better than Pearson. Stemming has inconsistent results, slightly lowering (2) but increasing (1)

Overall, Vader sentiment analysis produces very weak correlation with both (1) and (2). Try with other models.

Removing headlines from news sources with less articles:
Decreased correlation. Most of the correlations are negative but close to 0. This suggests that news sources with lower article counts tend to predict the price movements more accurately than the news sources that post more often.


Note: 

1. Might be helpful to determine the most relevant news sources by taking highly correlated news sources with instances of more than 20. Limiting the data might increase correlation. Use test set to evaluate if using this method.

2. Doing linear regression on neg, neu and pos score might produce interesting results.

3. Vader sentiment scores might be the problem. Sentiment scores of some sample headlines was observed and Vader had many false negatives. Try using article content.

In [ ]:
df_final_unstem.loc[df_final_unstem['news']==' The Telegraph', ]

Notes:

1. Take note of changes in the composition of S&P 500.